# How to spread points?

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import scienceplots
import pandas as pd
from itertools import combinations
import pprint
from shapely.geometry import Polygon
plt.style.use(['science', 'grid', 'ieee'])

from utils import (
    load_dataset,
    get_images,
    transform_points,
    get_densities,
    predict,
    calculate_errors,
    get_result_dict,
    SCALING_FACTOR,
    plot_setup, 
    plot_setup_noised,
    plot_predictions,
    add_outlier
)

def filter_points(points, keys):
    filtered = {key: points[key] for key in keys}
    return filtered

def conduct_experiment(pv_img, tv_img, plot=True, print_errors=True, used_ref_points=None, return_homography=False):
    # Load Data
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    img_pv, img_tv = get_images(pv_img, tv_img)
    
    if used_ref_points:
        reference_pts_pv = filter_points(reference_pts_pv, used_ref_points)
        reference_pts_tv = filter_points(reference_pts_tv, used_ref_points)
        
    # Transform Points to Homogeneous Numpy arrays
    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)
    # Calculate Homography
    h, _ = cv2.findHomography(
        reference_pts_pv_arr,
        reference_pts_tv_arr,
        # method = cv2.RANSAC,
        method = 0,
    )
    h_inv = np.linalg.inv(h)

    # Get Pixel Densities
    reference_densities = get_densities(reference_pts_tv_arr, h_inv)
    validation_densities = get_densities(validation_pts_tv_arr, h_inv)
    
    # Predict Points
    predicted_reference_pts_tv = predict(reference_pts_pv_arr, h)
    predicted_validation_pts_tv = predict(validation_pts_pv_arr, h)
    
    # Errors
    reference_errors = calculate_errors(predicted_reference_pts_tv, reference_pts_tv_arr)
    validation_errors = calculate_errors(predicted_validation_pts_tv, validation_pts_tv_arr)

    # Combine Results in Dictionary
    reference_result_dict = get_result_dict(reference_pts_pv, reference_pts_tv_arr, predicted_reference_pts_tv, reference_errors, reference_densities, reference_pts_pv_arr)
    validation_result_dict = get_result_dict(validation_pts_pv, validation_pts_tv_arr, predicted_validation_pts_tv, validation_errors, validation_densities, reference_pts_pv_arr)
    
    # Print Errors
    if print_errors:
        print(f'Reference  Error: {np.mean(reference_errors) :.2f} PX!')
        print(f'Reference  Error: {np.mean(reference_errors) / SCALING_FACTOR * 100 :.2f} CM!')
        print('---')
        print(f'Validation Error: {np.mean(validation_errors):.2f} PX!')
        print(f'Validation Error: {np.mean(validation_errors) / SCALING_FACTOR * 100 :.2f} CM!')
    
    # Plots
    if plot:
        plot_setup(img_tv, img_pv, reference_result_dict, validation_result_dict)
        plot_predictions(img_tv, img_pv, reference_result_dict, validation_result_dict)
    if return_homography:
        return reference_result_dict, validation_result_dict, h
    return reference_result_dict, validation_result_dict

def get_collinearity_score(subset, reference_pts_pv):
    A = np.array(reference_pts_pv[subset[0]])
    B = np.array(reference_pts_pv[subset[1]])
    C = np.array(reference_pts_pv[subset[2]])
    AB = np.linalg.norm(A-B) 
    AC = np.linalg.norm(A-C) 
    BC = np.linalg.norm(B-C)
    distances = sorted([AB, AC, BC])
    return (distances[0] + distances[1] - distances[2]) / distances[2] * 100
    
def get_overall_collinearity_score(used_ref_pts, reference_pts_pv):
    scores = []
    for subset in combinations(used_ref_pts, 3):
        score = get_collinearity_score(subset, reference_pts_pv)
        scores.append(score)
    return min(scores)

def get_collinearity_scores(used_ref_pts, reference_pts_pv):
    scores = []
    for subset in combinations(used_ref_pts, 3):
        score = get_collinearity_score(subset, reference_pts_pv)
        scores.append(score)
    return scores

Let's examine the case for four reference points, as they are the minimum required. We have determined the following possible influencing factors:
1. Collinearity
2. Distance to next reference point
3. Pixel Density
4. Spanned Area

## 1. Collinearity

**Conclusion**: Being almost collinear has some fatal consequences.

We need at least four points to determine a homography $\mathrm{H}$. It collapses when at least three points are collinear. In the real world, points are rarely collinear. However, we can quantify collinearity with the following measurement:

Given three points A, B, C and their corresponding edges a, b, c, let's calculate the following measure
$$
\text{collinearity} = (a + b - c) / c,
$$
where $c$ is the longest distance. Hence, given four reference points, we can determine $\begin{pmatrix} 4 \\ 3 \end{pmatrix} = 4$ collinearity scores. To quantify the collinearity score for the whole four reference point setup, we will take the smallest collinearity score. Note that this only works for four reference points. With more reference points, we might need to adjust it, as the homography has more chance to overcome the collinearity. INSERT MATH HERE FOR REFERENCE POINT SETUP.

In [ ]:
# Plot Functions
def plot_reference_point(name, x, y, ax, fc='white', ec='black', x_offset=-50, y_offset=0):
    ax.plot(x, y, marker='X', markersize=8, color='whitesmoke')
    ax.annotate(name, 
        xy=(x, y),
        color='black',
        fontsize=14,
        bbox=dict(facecolor=fc, edgecolor=ec, boxstyle="round", alpha=0.8, lw=1.5, pad=0.5)
    )
    
def plot_reference_points(reference_pts_pv, ax):
    for name, pt in reference_pts_pv.items(): 
        plot_reference_point(name, pt[0], pt[1], ax)

def plot_collinearity_score(R1, R2, R3, reference_pts_pv, ax, fc, ec):
    score = get_collinearity_score([R1, R2, R3], reference_pts_pv)
    A = reference_pts_pv[R1]
    B = reference_pts_pv[R2]
    C = reference_pts_pv[R3] 

    # Plot lines
    ax.plot([A[0], B[0], C[0]], [A[1], B[1], C[1]], lw=3, linestyle='--', color=fc)
    # Plot Score
    plot_reference_point(f'{score:.2f}', B[0]+50, B[1]-100, ax, 'white', ec)
    # Plot Reference points
    for name, pt in [(R1, A), (R2, B), (R3, C)]:
        plot_reference_point(name, pt[0], pt[1], ax, fc, ec)


def plot_collinearity_score_offset(R1, R2, R3, reference_pts_pv, ax, fc, ec, offset):
    score = get_collinearity_score([R1, R2, R3], reference_pts_pv)
    A = list(reference_pts_pv[R1])
    B = list(reference_pts_pv[R2])
    C = list(reference_pts_pv[R3])

    A[0] = A[0] + offset[0]
    B[0] = B[0] + offset[0]
    C[0] = C[0] + offset[0]
    
    A[1] = A[1] + offset[1]
    B[1] = B[1] + offset[1]
    C[1] = C[1] + offset[1]

    # Plot lines
    ax.plot([A[0], B[0], C[0]], [A[1], B[1], C[1]], lw=3, linestyle='dotted', color=fc)
    # Plot Score
    plot_reference_point(f'{score:.2f}', B[0]+0, B[1]-190, ax, 'white', fc)
    # Plot Reference points
    for name, pt in [(R1, A), (R2, B), (R3, C)]:
        plot_reference_point(name, pt[0], pt[1], ax, fc, ec)

def plot_used_reference_points(ref_pts, reference_pts_pv, ax):
    for name, pt in reference_pts_pv.items():
        if name not in ref_pts:
            continue
        plot_reference_point(name, pt[0], pt[1], ax, 'salmon', 'red')
        
# Necessary Variables
pv_img_fname = 'IMG_01'
tv_img_fname = 'IMG_00'

pv_img, tv_img = get_images(pv_img_fname, tv_img_fname)
reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img_fname, tv_img_fname)

In [ ]:
# Plot individual collinearity scores
fig, ax = plt.subplots(1, 1, figsize=(12, 10))

ax.imshow(pv_img, alpha=0.1)
ax.axis('off')

# plot_reference_points(reference_pts_pv, ax)
plot_collinearity_score_offset('w8', 'w9', 'w19', reference_pts_pv, ax, 'steelblue', 'royalblue', [100, 100])
plot_collinearity_score_offset('w4', 'w5', 'w7', reference_pts_pv, ax, 'salmon', 'tomato', [0, 0])
plot_collinearity_score_offset('w7', 'w6', 'w8', reference_pts_pv, ax, 'mediumseagreen', 'seagreen', [50, 50])

plt.savefig('output/collinearity_examples.png', dpi=100)
plt.show()

In [ ]:
# Plot Combined Collinarity score
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
ax.imshow(pv_img)
ax.axis('off')



plot_reference_points(reference_pts_pv, ax)
used_reference_points = ['w4', 'w5', 'w7', 'w8']
plot_used_reference_points(used_reference_points, reference_pts_pv, ax)
collinearity_score = get_overall_collinearity_score(used_reference_points, reference_pts_pv)
print(f'collinearity score: {collinearity_score:.2f}')

plt.show()

In [ ]:
def get_collinearity_scores_new(used_ref_pts, reference_pts_pv):
    scores = []
    for subset in combinations(used_ref_pts, 3):
        score = get_collinearity_score(subset, reference_pts_pv)
        scores.append(score)
    return scores

all_reference_pts = list(reference_pts_pv.keys())
result = {}
for subset in combinations(all_reference_pts, 3):
    score = get_collinearity_scores_new(subset, reference_pts_pv)
    result[subset] = score
result = {k: v for k, v in sorted(result.items(), key=lambda item: item[1])}
pprint.pp(result)

Now that the collinearity score seems to work, let's examine how it affects the results. We will use every possible combination of four reference points to calculate $\mathrm{H}$ and determine the collinearity score and the error. Then we can plot the results.

In [ ]:
def filter_points(points, keys):
    filtered = {key: points[key] for key in keys}
    return filtered

def collinearity_experiment(pv_img, tv_img):
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    all_reference_points = list(reference_pts_pv.keys())

    rows = []
    k=4
    # For all reference point combinations with k=4
    for i, subset in enumerate(combinations(all_reference_points, k)):
        print('#'*10)
        print(f'Experiment {i}')
        # Conduct Experiment
        reference_result_dict, validation_result_dict = conduct_experiment(pv_img, tv_img, used_ref_points=subset, plot=False)

        # Make resulting Dict 
        collinearity_score = get_overall_collinearity_score(subset, reference_pts_pv)
        for pt, pt_dict in validation_result_dict.items():
            pt_dict['k'] = 4
            pt_dict['experiment'] = i
            pt_dict['name'] = pt
            pt_dict['collinearity'] = collinearity_score
            pt_dict['used_ref_pts'] = subset
            rows.append(pt_dict)
    df = pd.DataFrame(rows)
    df['img'] = pv_img
    return df

result_df = collinearity_experiment('IMG_01', tv_img_fname) 

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

result_df.plot.scatter(
    x='collinearity', 
    y='error',
    s=200,
    c='distance_to_next_ref_pt',
    colormap='viridis',
    alpha=0.7,
    ax=ax1
)
result_df.loc[result_df['collinearity'] > 1].plot.scatter(
    x='collinearity', 
    y='error',
    s=200,
    c='distance_to_next_ref_pt',
    colormap='viridis',
    alpha=0.7,
    ax=ax2
)
# plt.savefig('collinearity_error.png', dpi=300)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)#, figsize=(12, 10))

result_df.plot.scatter(
    x='collinearity', 
    y='error',
    s=10,
   # c='#FDE725FF',
     c='#440154FF',
    # colormap='viridis',
    alpha=0.5,
    ax=ax1
)
result_df.loc[result_df['collinearity'] > 1].plot.scatter(
    x='collinearity', 
    y='error',
    s=10,
    #c='#440154FF',
    c='#22A884FF',
    # colormap='viridis',
    alpha=0.5,
    ax=ax2
)

ax1.set_ylabel('error [px]')
ax2.set_ylabel('error [px]')
ax2.set_xlabel('collinearity score')
ax1.grid(False)
ax2.grid(False)
plt.savefig('output/collinearity_error.png', dpi=600)
plt.show()

In [ ]:
outliers = result_df.loc[result_df['collinearity'] < 1]
inliers = result_df.loc[result_df['collinearity'] >=1]

In [ ]:
outlier_keys = outliers['used_ref_pts'].unique()

print('-----'*20)
print(f'Get rid of the following {len(outlier_keys)} Homographies with a collinearity score < 1:')
print('-----'*20)
pprint.pp(outlier_keys)
print('-----'*20)

In [ ]:
outliers.groupby(['experiment', 'used_ref_pts']).agg(
    mean=('error', lambda x: np.mean(x).round(2)),
    min=('error', lambda x: np.min(x).round(2)),
    max=('error', lambda x: np.max(x).round(2)),
    collinearity_score=('collinearity', lambda x: np.min(x).round(2))
)

What's the average score of the inliers?

In [ ]:
print(f'Mean inlier error: {inliers["error"].mean():.2f}')

In [ ]:
inliers.plot.scatter(
    x='collinearity', 
    y='error',
    s=100,
    c='pixel_density',
    colormap='viridis',
    alpha=0.8,
    figsize=(12, 10)
)
plt.show()

## 2. Distance to next Reference Point

**Conclusion**: Of course it influences the result. 

When a reference point was actually used, then of course near points should perform better, as the $Z$ Coordinate should **usually** be similar. When we use more than four points to calculate $\mathrm{H}$, then it is at least considered, however, depending on the used optimization technique. You will see an image about this under **4 Spanned Area**.

## 3. Pixel Density

**Conclusion:** It seems to really have some minor influence, but of course it's only one of many. I think it will have much more influence when we make the noise experiments, as we calculate the distance within the top view. See later section.

Validation Points that lie further away from the camera position have less pixels available around it, such that being a single pixel off, results in a higher error for the validation point. 

We can quantify pixel density as follows: Consider four points around the respective validation point within the top view, each one meter away in the x and y direction (positive and negative). The resulting four surrounding points can be transformed into the perspective view using the Homography $\mathrm{H}^{-1}$. We then average the distances between the points and the validation point within the perspective view.

In [ ]:
# Plot the Setup
pv_img_fname = 'IMG_02'
tv_img_fname = 'IMG_00'

pv_img, tv_img = get_images(pv_img_fname, tv_img_fname)
reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img_fname, tv_img_fname)
reference_result_dict, validation_result_dict = conduct_experiment('IMG_02', tv_img_fname, plot=False, print_errors=False)

plot_setup(tv_img, pv_img, reference_result_dict, validation_result_dict)

Now let's see if this has some influence on our results. Hence, we will calculate the homography matrix for every image using all reference points (no outliers) and then plot the errors of each validation points against the pixel density.

In [ ]:
def get_dataframe(pv_imgs, tv_img):
    dataframe = pd.DataFrame()
    rows = []
    for pv_img in pv_imgs:
        print(pv_img)
        reference_result_dict, validation_result_dict = conduct_experiment(pv_img, tv_img, plot=False, print_errors=False)
        for pt, pt_dict in reference_result_dict.items():
            pt_dict['name'] = pt
            pt_dict['img'] = pv_img
            pt_dict['point_type'] = 'reference'
            rows.append(pt_dict) 
        for pt, pt_dict in validation_result_dict.items():
            pt_dict['name'] = pt
            pt_dict['img'] = pv_img
            pt_dict['point_type'] = 'validation' 
            rows.append(pt_dict) 
    dataframe = pd.DataFrame(rows)  
    return dataframe

dataframe = get_dataframe(['IMG_01', 'IMG_02', 'IMG_03', 'IMG_04', 'IMG_05', 
                           'IMG_06', 'IMG_07', 'IMG_08', 'IMG_09', 'IMG_10', 
                           'IMG_11', 'IMG_12'], 'IMG_00')

Overall Error Distribution per image:

In [ ]:
validation_errors_per_img = dataframe[dataframe['point_type']=='validation'] 
validation_errors_per_img.boxplot(column='error', by='img', 
                                  vert=False, 
                                  patch_artist=True, 
                                  boxprops=dict(facecolor='lightgray', lw=1), 
                                  medianprops=dict(color='black', lw=2),
                                  whiskerprops = dict(color = "black", lw=1),
                                  widths=0.5,
                                  figsize=(15, 10))
plt.show()

Now how about pixel densities:

In [ ]:
# WHERE IS THE 2K??!
# COLOR PER IMAGE WOULD BE NICE
image_color_dict = {
    'IMG_01':1,  
    'IMG_02':2, 
    'IMG_03':3,  
    'IMG_04':4,  
    'IMG_05':5, 
    'IMG_06':6, 
    'IMG_07':7, 
    'IMG_08':8, 
    'IMG_09':9, 
     'IMG_10':10,
     'IMG_11':11,
     'IMG_12':12
}

validation_errors_per_img['color'] = validation_errors_per_img.apply(lambda x: image_color_dict[x['img']], axis=1)
validation_errors_per_img.plot.scatter(x='pixel_density',
                y='error',
                c='color',
                # s='distance_to_next_ref_pt', # TODO
                colormap='viridis',
                alpha=0.8,
                figsize=(15, 10)
)
plt.show()

Where is the 2000 density outlier? Validaiton point **b1** in **DJI_0053**. Let's get rid of him, s.t. we can see the plot better:

In [ ]:
validation_errors_per_img_wo_outlier = validation_errors_per_img.loc[validation_errors_per_img['pixel_density'] < 1000]
validation_errors_per_img_wo_outlier.plot.scatter(x='pixel_density',
                y='error',
                c='color',
                # s='distance_to_next_ref_pt', # TODO
                colormap='viridis',
                alpha=0.8,
                figsize=(15, 10)
)
plt.show()

## Spanned Area

**Conclusion:** The spanned area correlates strongly with the distance to the next reference point. This influences the error.

The bigger the spanned area, the more we average over the whole scene and can take more area into account. This strongly correlates to the distance to the next reference point of course.

In [ ]:
# Plot the Setup
pv_img_fname = 'IMG_01'
tv_img_fname = 'IMG_00'

pv_img, tv_img = get_images(pv_img_fname, tv_img_fname)
reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img_fname, tv_img_fname)
reference_result_dict, validation_result_dict = conduct_experiment('IMG_00', tv_img_fname, plot=False, print_errors=False)

Let's conduct the experiment with four reference points, but filter the collinear homographies for one specific image, e.g., DJI_0026.

In [ ]:
def filter_points(points, keys):
    filtered = {key: points[key] for key in keys}
    return filtered

def collinearity_experiment(pv_img, tv_img):
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    all_reference_points = list(reference_pts_pv.keys())

    rows = []
    k=4
    # For all reference point combinations with k=4
    for i, subset in enumerate(combinations(all_reference_points, k)):
        print('#'*10)
        print(f'Experiment {i}')
        # Conduct Experiment
        reference_result_dict, validation_result_dict = conduct_experiment(pv_img, tv_img, used_ref_points=subset, plot=False)

        # Make resulting Dict 
        collinearity_score = get_overall_collinearity_score(subset, reference_pts_pv)
        spanning_points = []
        for ref_pt_name in subset:
            spanning_points.append(reference_pts_pv[ref_pt_name])
        polygon = Polygon(spanning_points)
        spanned_area = polygon.area / (4000*3000) * 100
        for pt, pt_dict in validation_result_dict.items():
            pt_dict['k'] = 4
            pt_dict['experiment'] = i
            pt_dict['name'] = pt
            pt_dict['collinearity'] = collinearity_score
            pt_dict['used_ref_pts'] = subset
            pt_dict['spanned_area'] = spanned_area
            rows.append(pt_dict)
    df = pd.DataFrame(rows)
    df['img'] = pv_img
    return df

result_df = collinearity_experiment('IMG_01', tv_img_fname) 

In [ ]:
# Plot example area and the area score.
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 14))
ax1.imshow(pv_img)
ax1.axis('off')
ax2.imshow(pv_img)
ax2.axis('off')
plot_reference_points(reference_pts_pv, ax1)
plot_reference_points(reference_pts_pv, ax2)

used_reference_points = ['w4', 'w8', 'w5', 'w6']
plot_used_reference_points(used_reference_points, reference_pts_pv, ax1)
area = result_df.loc[result_df['used_ref_pts'] == tuple(used_reference_points)]['spanned_area'].values[0]
print(f'Area: {area:.2f}')

used_reference_points = ['w19', 'w21', 'w22', 'w9']
plot_used_reference_points(used_reference_points, reference_pts_pv, ax2)
area = result_df.loc[result_df['used_ref_pts'] == tuple(used_reference_points)]['spanned_area'].values[0]
print(f'Area: {area:.2f}')

plt.tight_layout()
plt.show()

In [ ]:
inliers = result_df.loc[result_df['collinearity'] >=1]
scatter = inliers.plot.scatter(
    x='spanned_area', 
    y='error',
    s=10,
    c='distance_to_next_ref_pt',
    colormap='viridis',
    alpha=0.8,
    colorbar=False
    # figsize=(12, 10)23
)
cbar = plt.colorbar(scatter.collections[0], ax=scatter)
# cbar.ax.set_title('distance to next reference point', ha='center', va='center', rotation=270)
cbar.set_label('distance to next reference point [px]')
plt.grid(False)
plt.ylabel('error [px]')
plt.xlabel('spanned area [px]')
plt.savefig('output/area_distance_error.png', dpi=600)
plt.show()